### Purpose of this notebook
The purpose of this notebook is to download WLASL dataset and upload to S3 bucket after processing as Numpy

The videos will be labeled by the 'gloss' name from the Json.
Some videos only have the required sign between Frame-A and Frame-B. These will be cropped accordingly

In [1]:
#Install missing packages
#!pip install boto3
#!pip install pyarrow
#!pip install fastparquet
#!pip install s3fs
#!pip install mediapipe
#!pip install kaggle

In [2]:
# Import required libraries
import boto3 #Video files get read through this
import cv2
import os
import pandas as pd
import math
import numpy as np
import random
import io
import s3fs # Parquet files get read through this
import zlib # For compression
import time # To calculate download time
import configparser
import requests
import psutil # Checks memory usage
import tempfile
import json
#import mediapipe as mp
from tqdm import tqdm

In [3]:
# Let's read in the credentials file
config = configparser.ConfigParser()
config.read('/home/ec2-user/.aws/credentials')

['/home/ec2-user/.aws/credentials']

In [4]:
aws_access_key_id = config["default"]['aws_access_key_id']
aws_secret_access_key = config["default"]['aws_secret_access_key']
bucket_name = 'asl-capstone'
prefix = '/WLASL/'
save_path = '/content/temp_folder'
s3_URI = 's3://asl-capstone/'

In [5]:
# Create an s3 object
s3 = boto3.client('s3',aws_access_key_id = aws_access_key_id, aws_secret_access_key = aws_secret_access_key,
                  region_name = 'us-west-2')

In [6]:
# Create an S3FS object
fs = s3fs.S3FileSystem(key=aws_access_key_id, secret=aws_secret_access_key) # Define an S3FS object

In [50]:
wlasl_files = fs.ls('s3://asl-capstone/wlasl/')
len(wlasl_files)

2017

In [51]:
all_parquet_files = [file for file in wlasl_files if 'parquet' in file]
print(len(all_parquet_files))

17


In [52]:
tmp_parq = pd.DataFrame()
for file in all_parquet_files:
    tmp_parq = pd.concat([tmp_parq, pd.read_parquet('s3://' + file)])

# tmp_parq.to_parquet('s3://asl-capstone/wlasl/video_mappings_masterfile.parquet')

In [69]:
tmp_parq.shape

(11938, 3)

In [54]:
wlasl_files_cleaned = [file for file in wlasl_files if '_' in file and '.npy' in file]
len(wlasl_files_cleaned)

3

In [55]:
vid_df = pd.DataFrame(columns=['filepath', 'word', 'frame_count'])

In [59]:
for file in tqdm(wlasl_files_cleaned):
    with fs.open(file, 'rb') as f:
        vid = np.load(f).astype('float32')
        
    word = file.split('_')[0].split('/')[-1]
    frames = len(vid)
    
    temp_df = pd.DataFrame([[file, word, frames]], columns=['filepath', 'word', 'frame_count'])
    
    vid_df = pd.concat([vid_df, temp_df])
    vid_df.to_parquet('s3://asl-capstone/wlasl/video_mappings_1.parquet')

    new_file_path = file.replace('/wlasl/', '/wlasl/RGB/')

    with fs.open(new_file_path, 'wb') as f:
        np.save(f, vid)

    fs.rm(file)
        

0it [00:00, ?it/s]
